In [2]:
import numpy as np
import pandas as pd
import torch
import random
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import numpy as np
import torch
import torch.nn as nn

def custom_batch_norm1d(input_tensor, eps):
    mean = torch.mean(input_tensor, dim=0)
    var = torch.var(input_tensor, dim=0)
    normed_tensor = (input_tensor-mean)/((var/2+eps)**0.5)
    return normed_tensor


input_tensor = torch.Tensor([[0.0, 0, 1, 0, 2], [0, 1, 1, 0, 10]])
batch_norm = nn.BatchNorm1d(input_tensor.shape[1], affine=False)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
import numpy as np
all_correct = True
for eps_power in range(10):
    eps = np.power(10., -eps_power)
    batch_norm.eps = eps
    batch_norm_out = batch_norm(input_tensor)
    custom_batch_norm_out = custom_batch_norm1d(input_tensor, eps)

    all_correct &= torch.allclose(batch_norm_out, custom_batch_norm_out)
    all_correct &= batch_norm_out.shape == custom_batch_norm_out.shape
print(all_correct)

True


In [4]:
import torch
import torch.nn as nn

input_size = 7
batch_size = 5
input_tensor = torch.arange(0,batch_size*input_size).reshape(batch_size, input_size).float()
eps = 1e-3

def custom_batch_norm1d(input_tensor, weight, bias, eps):
    mean = torch.mean(input_tensor, dim=0)
    var = torch.var(input_tensor, dim=0)
    var = (((input_tensor-mean)**2).sum(dim=0))/(len(input_tensor))
    normed_tensor = (input_tensor-mean)/((var+eps)**0.5)
    return normed_tensor*weight + bias

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm_out = batch_norm(input_tensor)

custom_batch_norm_out = custom_batch_norm1d(input_tensor, batch_norm.weight.data, batch_norm.bias.data, eps)
print(torch.allclose(batch_norm_out, custom_batch_norm_out) \
      and batch_norm_out.shape == custom_batch_norm_out.shape)

False


In [5]:
import torch
import torch.nn as nn


input_size = 3
batch_size = 5
eps = 1e-1


class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        self.weight = weight
        self.bias = bias
        self.eps = eps
        self.momentum = momentum
        self.running_mean = 0
        self.running_var = torch.ones(3)
        self.train = True
    def __call__(self, input_tensor):
        if self.train:
            mean = torch.mean(input_tensor, dim=0)
            var = (((input_tensor-mean)**2).sum(dim=0))/(len(input_tensor))
            var = torch.var(input_tensor, dim=0, unbiased=False)
            normed_tensor = (input_tensor-mean)/((var+self.eps)**0.5)
            self.running_mean = (1-self.momentum)*mean + self.momentum*self.running_mean
            self.running_var = 1.25*(1-self.momentum)*var + self.momentum*self.running_var
            return normed_tensor*self.weight + self.bias
        else:
            normed_tensor = (input_tensor-self.running_mean)/((self.running_var+self.eps)**0.5)
            return normed_tensor*self.weight + self.bias
    def eval(self):
        # В этом методе реализуйте переключение в режим предикта.
        self.train = False

batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data,
                                        batch_norm.bias.data, eps, batch_norm.momentum)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape

batch_norm.eval()
custom_batch_norm1d.eval()
# print(batch_norm.running_mean)
# print(custom_batch_norm1d.running_mean)
# print()
# print(batch_norm.running_var)
# print(custom_batch_norm1d.running_var)
# print()

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)

    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape
print(all_correct)

True


In [64]:
import torch
import torch.nn as nn

eps = 1e-3

input_channels = 3
batch_size = 3
height = 10
width = 10

batch_norm_2d = nn.BatchNorm2d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, height, width, dtype=torch.float)

def custom_batch_norm2d(input_tensor, eps):
    mean = torch.cat([torch.mean(input_tensor[:,i,:,:]).unsqueeze(0) for i in range(input_tensor.shape[1])], dim=0)
    var = torch.cat([torch.var(input_tensor[:,i,:,:], unbiased=False).unsqueeze(0) for i in range(input_tensor.shape[1])], dim=0)
    normed_tensor = torch.cat([((input_tensor[:,i,:,:] - mean[i])/torch.sqrt(var[i]+eps)).unsqueeze(1) for i in range(input_tensor.shape[1])], dim=1)
    return normed_tensor

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = batch_norm_2d(input_tensor)
custom_output = custom_batch_norm2d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output) and norm_output.shape == custom_output.shape)


True


In [65]:
import torch
import torch.nn as nn


eps = 1e-10


def custom_layer_norm(input_tensor, eps):
    shape = input_tensor.shape
    normed_tensor = input_tensor.clone()
    for i in range(shape[0]):
        mean = torch.mean(input_tensor[i])
        var = torch.var(input_tensor[i], unbiased=False)
        normed_tensor[i] = (input_tensor[i]-mean)/((var-eps)**0.5)
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for dim_count in range(3, 9):
    input_tensor = torch.randn(*list(range(3, dim_count + 2)), dtype=torch.float)
    layer_norm = nn.LayerNorm(input_tensor.size()[1:], elementwise_affine=False, eps=eps)

    norm_output = layer_norm(input_tensor)
    custom_output = custom_layer_norm(input_tensor, eps)

    all_correct &= torch.allclose(norm_output, custom_output, 1e-2)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

True


In [15]:
import torch
import torch.nn as nn

eps = 1e-3

batch_size = 5
input_channels = 2
input_length = 30

instance_norm = nn.InstanceNorm1d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, input_length, dtype=torch.float)


def custom_instance_norm1d(input_tensor, eps):
    mean = input_tensor.mean(dim=2, keepdim=True)
    var = input_tensor.var(dim=2, unbiased=False, keepdim=True)
    return (input_tensor - mean) / torch.sqrt(var + eps) * 1 + 0


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = instance_norm(input_tensor)
custom_output = custom_instance_norm1d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output, atol=1e-06) and norm_output.shape == custom_output.shape)


True


In [23]:
import torch
import torch.nn as nn

channel_count = 6
eps = 1e-3
batch_size = 20
input_size = 2

input_tensor = torch.randn(batch_size, channel_count, input_size)


def custom_group_norm(input_tensor, groups, eps):
    shape = input_tensor.shape
    input_tensor = input_tensor.reshape(shape[0], groups, -1)
    mean = torch.mean(input_tensor, dim=2, keepdim=True)
    var = torch.var(input_tensor, dim=2, keepdim=True, unbiased=False)
    normed_tensor = (input_tensor-mean)/((var+eps)**0.5)
    return normed_tensor.reshape(shape)


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for groups in [1, 2, 3, 6]:
    group_norm = nn.GroupNorm(groups, channel_count, eps=eps, affine=False)
    norm_output = group_norm(input_tensor)
    custom_output = custom_group_norm(input_tensor, groups, eps)
    all_correct &= torch.allclose(norm_output, custom_output, 1e-3)
    all_correct &= norm_output.shape == custom_output.shape
    break
print(all_correct)

True
